# Eat Safe, Love

## Notebook Set Up

In [12]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [13]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [14]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [18]:
# review the collections in our database
db.list_collection_names()

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65ffa3330bbbc474c8e9c29d, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [10]:
establishments.find_one()

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65ffa0ed0bbbc474c8e9c29c, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [9]:
# Find the establishments with a hygiene score of 20
query ={"scores.Hygiene": 20}
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
pprint(establishments.find_one(query))


KeyboardInterrupt: 

In [ ]:
# Convert the result to a Pandas DataFrame
results_df = pd.json_normalize(establishments.find(query))
results_df


# Display the number of rows in the DataFrame
print(results_df.count())

# Display the first 10 rows of the DataFrame
results_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
establishments.find_one()

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': "London"}, 'RatingValue': {'$gte': 4}}

results = establishments.find(query)

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)


# Display the first document in the results using pprint
pprint(establishments.find_one(query))



In [ ]:
# Convert the result to a Pandas DataFrame
results_df = pd.json_normalize(establishments.find(query))
results_df


# Display the number of rows in the DataFrame
print(results_df.count())


# Display the first 10 rows of the DataFrame
results_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

establishments.find_one({'BusinessName': 'Penang Flavours'})

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
         'geocode.latitude': {'$gt': latitude - degree_search, '$lt': latitude + degree_search},
         'geocode.longitude': {'$gt': longitude - degree_search, '$lt': longitude + degree_search},
         'RatingValue': 5
         }
            
sort = [('scores.Hygiene', -1)]

# Print the results
results = list(establishments.find(query).sort(sort))
pprint(results)

In [ ]:
# Convert result to Pandas DataFrame
results_df = pd.json_normalize(results)
results_df

# Display the number of rows in the DataFrame
print(results_df.count())

# Display the first 10 rows of the DataFrame
results_df.head(10)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0

match_query = match_query = {'$match': {'scores.Hygiene': 0}}

# 2. Groups the matches by Local Authority

group_query = {'$group': {'_id': {"LocalAuthorityName": "$LocalAuthorityName"}, 'count': { '$sum': 1 }}}
              
# 3. Sorts the matches from highest to lowest

sort_values = {'$sort': { 'count': -1 }}

# Print the number of documents in the result

pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame

results_df = pd.json_normalize(results)
results_df

# Display the number of rows in the DataFrame
print(results_df.count())

# Display the first 10 rows of the DataFrame
results_df.head(10)
